In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms

BATCH_SIZE   = 128
NUM_EPOCHS   = 1000
LR           = 1e-2
VAL_SPLIT    = 0.1
DEVICE       = torch.device("cuda" if torch.cuda.is_available() else "cpu")
SEED         = 120
torch.manual_seed(SEED)

In [ ]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import joblib


df = pd.read_csv("spotify_final_cleaned.csv",
                 header=None,
                 parse_dates=[21],
                 date_parser=lambda x: pd.to_datetime(x, format="%Y-%m-%d", errors="coerce"))

df = df[df[21] <= "2022-09-01"]

df = df.apply(pd.to_numeric, errors="coerce")
# Remove first 5 rows
df = df.dropna(subset=[5] + list(range(5, 19))).reset_index(drop=True)

# Pull out the features that works the best
cols = [15, 8, 11, 17, 14, 9, 6, 18, 16, 13]
X_numpy = df.iloc[:, cols].values
y_numpy = df.iloc[:, 5].values.astype(np.float32) / 100.0   

# train_test_split
X_train, X_val, y_train, y_val = train_test_split(
    X_numpy, y_numpy, test_size=0.1, random_state=42
)

# Z-score
scaler = StandardScaler().fit(X_train)
X_train_z = scaler.transform(X_train)
X_val_z   = scaler.transform(X_val)


# Save scaler
joblib.dump(scaler, "scaler.pkl")

# NumPy to PyTorch Tensor
X_train_t = torch.tensor(X_train_z, dtype=torch.float32)
y_train_t = torch.tensor(y_train,   dtype=torch.float32)
X_val_t   = torch.tensor(X_val_z,   dtype=torch.float32)
y_val_t   = torch.tensor(y_val,     dtype=torch.float32) 

# Create DataLoader
train_ds = TensorDataset(X_train_t, y_train_t)
val_ds   = TensorDataset(X_val_t,   y_val_t)

BATCH_SIZE = BATCH_SIZE
train_loader = DataLoader(
    train_ds,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=2,
    pin_memory=True
)
val_loader = DataLoader(
    val_ds,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=2,
    pin_memory=True
)

print(f"Training Set: {len(train_ds)}  |  Testing Set: {len(val_ds)}")

# Check the data
for batch_X, batch_y in train_loader:
    print("Shepe of one batch of X：", batch_X.shape) 
    print("Shape of one batch of y：", batch_y.shape)
    break


In [4]:
import torch
import torch.nn as nn

# Define the model
class SimpleMLP(nn.Module):
    def __init__(self, input_dim=10, hidden1=256, hidden2=128, hidden3=64, hidden4=32, hidden5=12):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden1),
            nn.ReLU(),
            nn.BatchNorm1d(hidden1),
            nn.Dropout(0.2),
            nn.Linear(hidden1, hidden2),
            nn.ReLU(),
            nn.BatchNorm1d(hidden2),
            nn.Dropout(0.2),
            nn.Linear(hidden2, hidden3),
            nn.ReLU(),
            nn.BatchNorm1d(256),
            nn.Dropout(0.15),
            nn.Linear(hidden3, hidden4),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Dropout(0.1),
            nn.Linear(hidden4, hidden5),
            nn.ReLU(),
            nn.BatchNorm1d(64),
            nn.Dropout(0.1),
            nn.Linear(hidden5, 1),
            # nn.Sigmoid()
        )

    def forward(self, x):
        return self.net(x)

# Move the model oh to the DEVICE
model = SimpleMLP(input_dim=10, hidden1=512, hidden2=384, hidden3=256, hidden4=128, hidden5=64).to(DEVICE)
print(model)


SimpleMLP(
  (net): Sequential(
    (0): Linear(in_features=10, out_features=512, bias=True)
    (1): ReLU()
    (2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.2, inplace=False)
    (4): Linear(in_features=512, out_features=384, bias=True)
    (5): ReLU()
    (6): BatchNorm1d(384, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): Dropout(p=0.2, inplace=False)
    (8): Linear(in_features=384, out_features=256, bias=True)
    (9): ReLU()
    (10): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): Dropout(p=0.15, inplace=False)
    (12): Linear(in_features=256, out_features=128, bias=True)
    (13): ReLU()
    (14): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (15): Dropout(p=0.1, inplace=False)
    (16): Linear(in_features=128, out_features=64, bias=True)
    (17): ReLU()
    (18): BatchNorm1d(64, eps=1e-05, momentum=0.1, affi

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define loss
criterion = nn.SmoothL1Loss(beta=0.5)

# Adam optimizer
optimizer = optim.Adam(model.parameters(), lr=LR, weight_decay=1e-5)

# ReduceLROnPlateau, lr * "factor" when loss stop reduce
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.2, patience=3, min_lr=1e-6)

# Training function
def train_one_epoch(epoch):
    model.train()
    running_loss = 0.0
    total_samples = 0

    for batch_X, batch_y in train_loader:
        batch_X = batch_X.to(DEVICE)   
        batch_y = batch_y.to(DEVICE)    
        
        # Forwarding 
        optimizer.zero_grad()
        output = model(batch_X)             # 输出形状 [batch_size, 1]
        preds = output.squeeze(1)           # 变成 [batch_size]
        loss = criterion(preds, batch_y)

        # Backwarding
        loss.backward()
        optimizer.step()

        # Loss
        running_loss += loss.item() * batch_X.size(0)
        total_samples += batch_X.size(0)

    avg_loss = running_loss / total_samples
    print(f"[Epoch {epoch:02d}/{NUM_EPOCHS}]  Training Set Loss: {avg_loss:.4f}")
    return avg_loss


@torch.no_grad()
def evaluate(loader, mode="Val"):
    model.eval()
    total_loss = 0.0
    total_samples = 0

    for batch_X, batch_y in loader:
        batch_X = batch_X.to(DEVICE)
        batch_y = batch_y.to(DEVICE)

        output = model(batch_X)
        preds = output.squeeze(1)
        loss = criterion(preds, batch_y)

        total_loss += loss.item() * batch_X.size(0)
        total_samples += batch_X.size(0)

    avg_loss = total_loss / total_samples
    print(f"[{mode}]  Sample Number: {total_samples}  Loss: {avg_loss:.4f}")
    return avg_loss

# Training
best_val_loss = float("inf")

for epoch in range(1, NUM_EPOCHS + 1):
    # —— Train one epoch —— #
    train_loss = train_one_epoch(epoch)

    # —— Update Learning Rate —— #
    scheduler.step(train_loss)

    # —— Evaluate test set every 10 epoch —— #
    if epoch % 10 == 0:
        val_loss = evaluate(val_loader, mode="Val")
        
    torch.save(model.state_dict(), "best_model.pth")



[Epoch 01/1000]  训练集 Loss: 0.0276
[Epoch 02/1000]  训练集 Loss: 0.0276
[Epoch 03/1000]  训练集 Loss: 0.0275
[Epoch 04/1000]  训练集 Loss: 0.0275
[Epoch 05/1000]  训练集 Loss: 0.0275
[Epoch 06/1000]  训练集 Loss: 0.0275
[Epoch 07/1000]  训练集 Loss: 0.0275
[Epoch 08/1000]  训练集 Loss: 0.0267
[Epoch 09/1000]  训练集 Loss: 0.0264
[Epoch 10/1000]  训练集 Loss: 0.0264
[Val]  样本数: 5763  Loss: 0.0269
    >>> 验证集 Loss 新低，已保存模型：best_model.pth
[Epoch 11/1000]  训练集 Loss: 0.0263
[Epoch 12/1000]  训练集 Loss: 0.0262
[Epoch 13/1000]  训练集 Loss: 0.0262
[Epoch 14/1000]  训练集 Loss: 0.0262
[Epoch 15/1000]  训练集 Loss: 0.0261
[Epoch 16/1000]  训练集 Loss: 0.0260
[Epoch 17/1000]  训练集 Loss: 0.0260
[Epoch 18/1000]  训练集 Loss: 0.0260
[Epoch 19/1000]  训练集 Loss: 0.0260
[Epoch 20/1000]  训练集 Loss: 0.0259
[Val]  样本数: 5763  Loss: 0.0265
    >>> 验证集 Loss 新低，已保存模型：best_model.pth
[Epoch 21/1000]  训练集 Loss: 0.0260
[Epoch 22/1000]  训练集 Loss: 0.0259
[Epoch 23/1000]  训练集 Loss: 0.0259
[Epoch 24/1000]  训练集 Loss: 0.0259
[Epoch 25/1000]  训练集 Loss: 0.0258
[Epoch

KeyboardInterrupt: 

In [7]:
import torch
import numpy as np

# -------------------------------------------------------------
# 假设以下变量已在前面定义并可用：
#   DEVICE      (torch.device)
#   model       (SimpleMLP 实例)
#   val_loader  (DataLoader，用于验证集，此处当作“测试集”)
#   "best_model.pth"  已保存的最优模型权重文件
# -------------------------------------------------------------

@torch.no_grad()
def evaluate_mae_r2_and_print_all():
    # 1. 加载最优权重并切换到评估模式
    model.load_state_dict(torch.load("best_model.pth"))
    model.to(DEVICE)
    model.eval()

    all_preds = []
    all_trues = []
    

    # 2. 遍历验证集，收集预测值和真实值
    for batch_X, batch_y in val_loader:
        batch_X = batch_X.to(DEVICE)    # [batch_size, 13]
        batch_y = batch_y.to(DEVICE)    # [batch_size]

        output = model(batch_X)         # [batch_size, 1]
        preds = output.squeeze(1)       # [batch_size]
        
        # 移回 CPU 并转换成 NumPy
        all_preds.append(preds.cpu().numpy())
        all_trues.append(batch_y.cpu().numpy())

    # 3. 拼接所有 batch
    all_preds = np.concatenate(all_preds, axis=0)  # 形状 (N,)
    all_trues = np.concatenate(all_trues, axis=0)  # 形状 (N,)
    
    # 4. 计算 MAE
    mae = np.mean(np.abs(all_preds - all_trues))

    # 5. 计算 R2：1 - SS_res / SS_tot
    ss_res = np.sum((all_trues - all_preds) ** 2)
    ss_tot = np.sum((all_trues - np.mean(all_trues)) ** 2)
    r2 = 1 - ss_res / ss_tot

    print(f"[Val-as-Test] numOfSamples: {len(all_trues)}")
    print(f"Mean Absolute Error (MAE): {mae:.4f}")
    print(f"R2 Score: {r2:.4f}")
    
    # 6. 打印所有真实值和预测值
    print("\n—— 真实值 (True) vs. 预测值 (Pred) ——")
    for true_val, pred_val in zip(all_trues, all_preds):
        print(f"True: {true_val:.4f}  |  Pred: {pred_val:.4f}")
    
    return all_trues, all_preds, mae, r2

# 调用并获取指标
all_trues, all_preds, mae_value, r2_value = evaluate_mae_r2_and_print_all()


[Val-as-Test] numOfSamples: 5254
Mean Absolute Error (MAE): 0.1249
R2 Score: 0.2242

—— 真实值 (True) vs. 预测值 (Pred) ——
True: 0.2300  |  Pred: 0.4428
True: 0.3200  |  Pred: 0.4344
True: 0.5600  |  Pred: 0.3389
True: 0.3500  |  Pred: 0.5514
True: 0.2600  |  Pred: 0.3982
True: 0.4700  |  Pred: 0.4401
True: 0.2600  |  Pred: 0.2437
True: 0.4100  |  Pred: 0.4791
True: 0.6400  |  Pred: 0.4394
True: 0.5600  |  Pred: 0.5022
True: 0.6300  |  Pred: 0.3768
True: 0.7100  |  Pred: 0.4830
True: 0.6200  |  Pred: 0.4994
True: 0.4600  |  Pred: 0.4119
True: 0.5200  |  Pred: 0.4667
True: 0.6900  |  Pred: 0.4457
True: 0.2000  |  Pred: 0.3448
True: 0.4900  |  Pred: 0.4019
True: 0.1400  |  Pred: 0.2019
True: 0.5700  |  Pred: 0.3991
True: 0.3600  |  Pred: 0.3262
True: 0.1600  |  Pred: 0.1874
True: 0.7800  |  Pred: 0.4020
True: 0.3600  |  Pred: 0.4607
True: 0.8200  |  Pred: 0.4944
True: 0.3100  |  Pred: 0.4189
True: 0.5200  |  Pred: 0.4321
True: 0.5800  |  Pred: 0.4335
True: 0.3800  |  Pred: 0.4254
True: 0.1600 